In [1]:
import os
import requests
import pandas as pd
import json

from pathlib import Path
import csv

from dotenv import load_dotenv
load_dotenv()

import pandas_datareader as web
import datetime as dt

%matplotlib inline

In [2]:
start = dt.datetime(2014,9,16)
end = dt.datetime.now()

btc_date = dt.datetime(2014,9,16)

iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

base_url = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'


token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')
params = {'token': token}
test_params = {'token': test_token}

test_resp = requests.get(base_url + 'status')
test_resp

<Response [200]>

In [3]:
def get_close(ticker):
    data = web.DataReader(ticker, 'yahoo', start, end)
    df = data[["Close"]]
    df.rename(columns={'Close':ticker},inplace=True)
    return df

def yahoo_10_year_data(tickers_list):
#     start = dt.datetime(2011,9,20)
#     end = dt.datetime.now()
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list:
        if i == 0: 
            raw_df_1 = web.DataReader(i, 'yahoo', start, end)
            combined_df_f = raw_df_1[['Close']]
            combined_df_f.rename(columns={'Close':i},inplace=True)
            
        else:
            raw_df_2 = web.DataReader(i, 'yahoo', start, end)
            df_f = raw_df_2[['Close']]
            df_f.rename(columns={'Close':i},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            #combined_df_f = combined_df_f.merge(df_f, how='inner',right_index=True, left_index=True)


    return combined_df_f

def test_iex_apicall_get_close_10_years(tickers_list):
    combined_df_f = pd.DataFrame()
    
    for i in tickers_list: 
        
        if i == 0: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/10y?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())
            
            resp_df.rename(columns={'date':'Date'},inplace=True)
            resp_df['Date'] = pd.to_datetime(resp_df['Date'])
            data = resp_df.set_index('Date')
            combined_df_f = data[['close']]
            symbol = i 
            combined_df_f.rename(columns={'close':symbol},inplace=True)
            
        else: 
            resp_data = requests.get(sandbox_url+'stock/'+i+'/chart/10y?token='+test_token)
            resp_df = pd.DataFrame(resp_data.json())
            
            resp_df.rename(columns={'date':'Date'},inplace=True)
            resp_df['Date'] = pd.to_datetime(resp_df['Date'])
            data = resp_df.set_index('Date')
            df_f = data[['close']]
            #symbol = data.iloc[0,4]
            symbol = i
            df_f.rename(columns={'close':symbol},inplace=True)
            
            combined_df_f = pd.concat([combined_df_f,df_f], axis='columns')
            
    return combined_df_f 

def call_test_chart_api(stock_ticker, chart_range):
    
    resp = requests.get(sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp.json())
    df.set_index('date',inplace=True)
    return df

def get_close(ticker):
    df = web.DataReader(ticker, 'yahoo', start, end)
    df = df[["Close"]]
    df.rename(columns={'Close':ticker},inplace=True)
    return df

In [4]:
iex_stock_df = pd.read_csv(
    Path("Resources/iex_stock_list.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    #index_col="Date",
)
iex_stock_df_date_index = pd.read_csv(
    Path("Resources/iex_stock_list.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    index_col="Date",
)

In [5]:
#aapl_df = call_test_chart_api(stock_ticker='AAPL', chart_range='73m')
# test call for 73months 

In [6]:
house_price = pd.read_csv(
    Path("Resources/combined_df.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    #index_col="Date",
)

In [7]:
house_value = house_price[["Date","House Value"]]

In [8]:
crypto_df = pd.read_csv(
    Path("Resources/crypto_df.csv"),
    infer_datetime_format=True,
    parse_dates=True,
    index_col="Date",
)

In [9]:
# test_df = iex_stock_df.iloc[::21,:]

vus_df = get_close("VUS.TO")
iex_stock_df_date_index = vus_df.merge(iex_stock_df_date_index, how='inner',right_index=True, left_index=True)

In [10]:
# Need to re-merge IEX and Crypto Data 

full_df = crypto_df.merge(iex_stock_df_date_index, how='inner',right_index=True, left_index=True)

In [11]:
full_df.reset_index(inplace=True)

In [12]:
1513/73

20.726027397260275

In [13]:
new_df = full_df.iloc[::21,:]

In [14]:
new_df.reset_index(inplace=True)
new_df.drop(columns={"index","Date"},inplace=True)

C:\Users\watso\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
final_df = pd.concat([house_value,new_df], axis="columns")
final_df.set_index("Date",inplace=True)

In [16]:
final_df

,House Value,BTC-USD,ETH-USD,VUS.TO,GME,AMC,AMZN,WMT,NVDA,AMD,AAPL,MSFT,DIS,PYPL
Date,,,,,,,,,,,,,,
2015-08-01,91.8,279.584991,2.772120,42.209999,46.96,30.27,551.97,75.18,5.2700,2.130,29.7585,48.31,111.55,40.33
2015-09-01,92.0,235.018997,1.338810,38.959999,41.95,28.64,508.00,66.21,5.6589,1.880,28.1079,43.89,102.54,35.03
2015-10-01,92.4,242.968994,0.609388,39.790001,43.69,26.63,548.34,66.51,6.7391,1.880,29.0136,47.71,107.11,32.36
2015-11-01,92.6,386.354004,0.895637,42.049999,49.00,27.64,685.98,59.95,7.0786,2.300,30.7300,56.95,116.00,37.83
2015-12-01,92.8,395.536011,0.811264,41.610001,33.16,24.51,673.20,63.30,8.3032,2.436,30.1900,57.26,116.22,36.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-01,118.3,37304.691406,2430.621338,78.820000,178.71,13.03,3312.08,145.25,150.8620,78.970,127.6900,250.47,175.52,254.52
2021-05-01,119.9,33723.027344,1989.736328,80.980003,222.80,59.45,3606.87,141.46,197.0699,83.890,137.3400,274.40,174.40,294.09
2021-06-01,120.7,34292.445312,2189.218750,83.430000,187.58,38.42,3807.86,146.96,200.5200,95.200,149.3100,294.29,180.19,315.84
